In [821]:
import numpy as np
import pandas as pd
from pandas import *
import csv

In [835]:
#create a list of csvs
files = !ls *.csv
#load into a df
df_list = [pd.read_csv(file) for file in files]
print files

['APP_data_reduced.csv', 'BB_data_reduced.csv', 'DS_data_reduced.csv', 'SLD_data_reduced.csv', 'SP_data_reduced.csv']


In [836]:
data = pd.concat(df_list)

In [837]:
data.head()

,id,rating,bitter,meaty,piquant,salty,sour,sweet,cookTimeInSeconds,ingredientCount,numberOfServings,prepTimeInSeconds,totalTimeInSeconds,ingredient_list,course
0,Bacon-Cheddar-Pinwheels-768341,4,0.833333,0.833333,0.000000,0.833333,0.166667,0.166667,NaN,5,16.0,900.0,2100.0,[Pillsbury™ Refrigerated Crescent Dinner Rolls...,Appetizer
1,Fiesta-Corn-Dip-1711704,4,0.500000,0.166667,0.833333,0.833333,0.500000,0.166667,NaN,7,4.0,900.0,900.0,"[sour cream, mayonnaise, ground cumin, mexicor...",Appetizer
2,Crunchy-Taco-Cups-1684027,4,0.833333,0.500000,0.166667,0.833333,0.166667,0.166667,NaN,5,12.0,NaN,2400.0,"[lean ground beef, taco seasoning, diced tomat...",Appetizer
3,Chipotle-Meatball-Appetizers-768376,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,24.0,1800.0,3000.0,[Pillsbury™ Refrigerated Crescent Dinner Rolls...,Appetizer
4,Sausage-Crescent-Cheese-Balls-768337,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,42.0,1200.0,2400.0,"[spicy sausage, shredded sharp cheddar cheese,...",Appetizer


In [838]:
# drop duplicates 
data = data.drop_duplicates('id')
data.shape

(9996, 15)

In [839]:
# labelencode the course column
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['course_num'] = le.fit_transform(data.course)

In [840]:
#make all column headers, small cases
data.columns = [col.lower() for col in data.columns]

In [841]:
data.isnull().sum()

id                       0
rating                   0
bitter                3233
meaty                 3233
piquant               3233
salty                 3233
sour                  3233
sweet                 3233
cooktimeinseconds     7397
ingredientcount          0
numberofservings         2
preptimeinseconds     6177
totaltimeinseconds       0
ingredient_list          0
course                   0
course_num               0
dtype: int64

In [842]:
data = data.drop(['cooktimeinseconds','preptimeinseconds'], axis = 1)

In [843]:
# #clean cook/preptime columns
# for index, row in data.iterrows():
#     if row['cooktimeinseconds'].isnull() and row['preptimeinseconds'].isnull():
#         print row
    

In [844]:
# clean the ingredient column 
import string
# strip non ascii characters 
printable = set(string.printable)
ingredient = [filter(lambda x: x in printable, z) for z in data['ingredient_list']]

In [845]:
ingredient[0]

'[Pillsbury Refrigerated Crescent Dinner Rolls, ranch dressing, bacon pieces, shredded cheddar cheese, green onions]'

In [846]:
# remove the braquets-convert to string
ingredient = [z[1:-1] for z in ingredient]

In [847]:
ingredient[0]

'Pillsbury Refrigerated Crescent Dinner Rolls, ranch dressing, bacon pieces, shredded cheddar cheese, green onions'

In [848]:
# Make each word a string
ingredient = [z.split(" ") for z in ingredient]

In [849]:
ingredient[0]

['Pillsbury',
 'Refrigerated',
 'Crescent',
 'Dinner',
 'Rolls,',
 'ranch',
 'dressing,',
 'bacon',
 'pieces,',
 'shredded',
 'cheddar',
 'cheese,',
 'green',
 'onions']

In [850]:
# perform word lemmatization
from nltk import stem
wordnet_lemm = stem.WordNetLemmatizer()

lem_ingredient = [] # create a new list of lemmatized ingredient list for each recipe
for ing in ingredient:
    lem_ingredient.append([wordnet_lemm.lemmatize(w) for w in ing])
     
# convert back to a string for vectorization
lem_ingredient = [' '.join(z) for z in lem_ingredient]
  

In [851]:
data['ingredient_lem'] = lem_ingredient
data = data.drop('ingredient_list', axis = 1)
data.head(2)

,id,rating,bitter,meaty,piquant,salty,sour,sweet,ingredientcount,numberofservings,totaltimeinseconds,course,course_num,ingredient_lem
0,Bacon-Cheddar-Pinwheels-768341,4,0.833333,0.833333,0.000000,0.833333,0.166667,0.166667,5,16.0,2100.0,Appetizer,0,"Pillsbury Refrigerated Crescent Dinner Rolls, ..."
1,Fiesta-Corn-Dip-1711704,4,0.500000,0.166667,0.833333,0.833333,0.500000,0.166667,7,4.0,900.0,Appetizer,0,"sour cream, mayonnaise, ground cumin, mexicorn..."


In [852]:
data.to_csv('data.csv')

In [867]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words='english')

In [868]:
array = data.values
from sklearn import cross_validation

X = data['ingredient_lem']
print X.shape
Y = data['course_num']
print Y.shape
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, random_state=10)

print X_train.shape
print X_test.shape

(9996,)
(9996,)
(7497,)
(2499,)


In [869]:
#vectorize and convert the matrix into array 
X_train_vect = vect.fit_transform(X_train) 
X_train_vect = X_train_vect.toarray() 
X_test_vect = vect.transform(X_test)
X_test_vect = X_test_vect.toarray()

In [870]:
#create a list of the feature names
features = vect.get_feature_names()
len(features)

1801

In [871]:
#create dataframe of the vectorized dataset
train_vect_df = pd.DataFrame(data = X_train_vect, columns = features)
train_vect_df['course_num'] = Y_train.values
test_vect_df = pd.DataFrame(data = X_test_vect, columns = features)
test_vect_df['course_num'] = Y_test.values
print train_vect_df.shape
print test_vect_df.shape

(7497, 1802)
(2499, 1802)


In [872]:
data.head(2)

,id,rating,bitter,meaty,piquant,salty,sour,sweet,ingredientcount,numberofservings,totaltimeinseconds,course,course_num,ingredient_lem
0,Bacon-Cheddar-Pinwheels-768341,4,0.833333,0.833333,0.000000,0.833333,0.166667,0.166667,5,16.0,2100.0,Appetizer,0,"Pillsbury Refrigerated Crescent Dinner Rolls, ..."
1,Fiesta-Corn-Dip-1711704,4,0.500000,0.166667,0.833333,0.833333,0.500000,0.166667,7,4.0,900.0,Appetizer,0,"sour cream, mayonnaise, ground cumin, mexicorn..."


In [873]:
# train/test split the other columns from the main dataframe 
array = data.values
_X = array[:, 1:11]
print _X.shape
Y = data['course_num']
print Y.shape
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(_X, Y, random_state=10)
print X_train.shape
print X_test.shape


(9996, 10)
(9996,)
(7497, 10)
(2499, 10)


In [874]:
# create a dataframe for training and test set
train_df = pd.DataFrame(data = X_train, columns = list(data.columns[1:11]))
test_df = pd.DataFrame(data = X_test, columns = list(data.columns[1:11]))
print train_df.shape 

(7497, 10)


In [875]:
# combine the dataframes for the training set and for the testing set. The new dataframes will have the vectorized
# columns and selected features together.
data_train = pd.concat([train_df, train_vect_df], axis = 1)
print data_train.shape
data_test = pd.concat([test_df, test_vect_df], axis = 1)
print data_test.shape

(7497, 1812)
(2499, 1812)


In [876]:
# save dfs to csv
data_train.to_csv('train.csv')
data_test.to_csv('test.csv')

In [866]:
data_train.head()

,rating,bitter,meaty,piquant,salty,sour,sweet,ingredientcount,numberofservings,totaltimeinseconds,...,zucchini snow,zucchini squash,zucchini string,zucchini sweet,zucchini tahini,zucchini tomatoes,zucchini vegetable,zucchini vidalia,zucchini yellow,course_num
0,4,NaN,NaN,NaN,NaN,NaN,NaN,5,1,900,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,4,0.833333,0.833333,0,0.833333,0.166667,0.166667,5,12,300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,4,0.833333,0.833333,0,0.833333,0.833333,0.333333,7,4,900,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,0.666667,0.666667,0,0.833333,0.166667,0.833333,11,12,3120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,4,NaN,NaN,NaN,NaN,NaN,NaN,15,12,5400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
